In [ ]:
!pip install --quiet optuna
import optuna

optuna.__version__

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.3 MB/s eta 0:00:00


'3.5.0'

In [ ]:
import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection


def objective():
    iris = sklearn.datasets.load_iris()  # Prepare the data.

    clf = sklearn.ensemble.RandomForestClassifier(n_estimators=5, max_depth=3)  # Define the model.

    return sklearn.model_selection.cross_val_score(
        clf, iris.data, iris.target, n_jobs=-1, cv=3
    ).mean()  # Train and evaluate the model.


print("Accuracy: {}".format(objective()))

Accuracy: 0.9533333333333333


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import csv
import numpy as np
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/dataset/breast-cancer.csv')
print(df.columns)
print(df.describe())


Mounted at /content/drive
Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'],
      dtype='object')
                 id  radius_mean  texture_mean  perimeter_mean    area_mean  \
count  5.690000e+02   569.000000    569.000000      569.000000   569.000000   
mean   3.037183e+07    14.127292     19.289649       91.969033   654.889104   
std    1.250206e+08     3.524049      4.301036       24.298981   351.914129   
min    8

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import optuna

X = df.drop(['diagnosis'], axis=1)
y = df['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 2, 20)
    max_depth = int(trial.suggest_float("max_depth", 1, 32, log=True))

    clf = sklearn.ensemble.RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)

    return sklearn.model_selection.cross_val_score(
        clf, X_train, y_train, n_jobs=-1, cv=3
    ).mean()


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

trial = study.best_trial

print("Accuracy: {}".format(trial.value))
print("Best hyperparameters: {}".format(trial.params))
model = sklearn.ensemble.RandomForestClassifier(n_estimators=int(trial.params['n_estimators']), max_depth=int(trial.params['max_depth']))
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
print("Train accuray: ", (y_pred == y_train).sum() / len(y_pred))
y_pred = model.predict(X_test)
print("Test accuray: ", (y_pred == y_test).sum() / len(y_pred))

[I 2024-03-02 06:27:36,945] A new study created in memory with name: no-name-d6a73ed2-aa4a-46b8-89af-90b717862344
[I 2024-03-02 06:27:37,172] Trial 0 finished with value: 0.9582316718949692 and parameters: {'n_estimators': 13, 'max_depth': 12.92147466920162}. Best is trial 0 with value: 0.9582316718949692.
[I 2024-03-02 06:27:37,356] Trial 1 finished with value: 0.9428807947019867 and parameters: {'n_estimators': 6, 'max_depth': 6.688760695546129}. Best is trial 0 with value: 0.9582316718949692.
[I 2024-03-02 06:27:37,463] Trial 2 finished with value: 0.9165069129778088 and parameters: {'n_estimators': 3, 'max_depth': 1.915294052632052}. Best is trial 0 with value: 0.9582316718949692.
[I 2024-03-02 06:27:37,702] Trial 3 finished with value: 0.960439177413733 and parameters: {'n_estimators': 12, 'max_depth': 7.704394176389323}. Best is trial 3 with value: 0.960439177413733.
[I 2024-03-02 06:27:37,833] Trial 4 finished with value: 0.9362728012083189 and parameters: {'n_estimators': 3, 'm

Accuracy: 0.960439177413733
Best hyperparameters: {'n_estimators': 12, 'max_depth': 7.704394176389323}
Train accuray:  1.0
Test accuray:  0.9473684210526315
